In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import sklearn_pandas
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from sklearn.neural_network import MLPClassifier

In [3]:
#preprocessed_data = pd.read_csv("clean_investmentdata_for_ml.csv",encoding='ISO-8859-1')
data_cleaned_new = pd.read_csv("filter_no_outliers.csv",encoding='ISO-8859-1')
print(data_cleaned_new.describe())

       funding_total_usd  funding_rounds  founded_year           seed  \
count       2.234200e+04    22342.000000  17128.000000   22342.000000   
mean        2.422386e+06        1.467684   2007.743111   28771.210008   
std         3.369504e+06        0.920213      7.753063   65717.822703   
min         1.000000e+00        1.000000   1902.000000       0.000000   
25%         1.430800e+05        1.000000   2006.000000       0.000000   
50%         8.755750e+05        1.000000   2010.000000       0.000000   
75%         3.600000e+06        2.000000   2012.000000   19299.000000   
max         2.425000e+07       14.000000   2014.000000  337471.000000   

            venture  equity_crowdfunding   undisclosed  convertible_note  \
count  2.234200e+04         2.234200e+04  2.234200e+04      2.234200e+04   
mean   1.752613e+06         9.233143e+03  8.252383e+04      1.549078e+04   
std    2.668400e+06         2.017313e+05  8.837722e+05      2.461842e+05   
min    0.000000e+00         0.000000e+

In [4]:
# correctly drop non-numeric columns that are still present in the dataset
columns_to_drop = ['permalink','name','homepage_url','category_list', 'market', 'status', 'country_code','state_code','region', 'city']
data_nn = data_cleaned_new.drop(columns=columns_to_drop, errors='ignore')

# convert the columns to datetime format
data_nn['first_funding_year'] = pd.to_datetime(data_nn['first_funding_at'], errors='coerce').dt.year
data_nn['last_funding_year'] = pd.to_datetime(data_nn['last_funding_at'], errors='coerce').dt.year

# drop the columns that have already been converted to years
data_nn = data_nn.drop(columns=['founded_at','first_funding_at', 'last_funding_at','founded_month','founded_qarter'], errors='ignore')

# one-hot encode the cleaned dataset again
data_nn = pd.get_dummies(data_nn, drop_first=True)

# display the shape and first few rows of the newly encoded data
data_nn.shape, data_nn.head()

((22342, 211),
    funding_total_usd  funding_rounds  founded_year     seed    venture  \
 0          4000000.0             2.0           NaN      0.0  4000000.0   
 1            40000.0             1.0        2012.0  40000.0        0.0   
 2            60000.0             2.0        2014.0      0.0        0.0   
 3          7000000.0             1.0        2011.0      0.0  7000000.0   
 4          2000000.0             1.0        2007.0      0.0  2000000.0   
 
    equity_crowdfunding  undisclosed  convertible_note  debt_financing  angel  \
 0                  0.0          0.0               0.0             0.0    0.0   
 1                  0.0          0.0               0.0             0.0    0.0   
 2              60000.0          0.0               0.0             0.0    0.0   
 3                  0.0          0.0               0.0             0.0    0.0   
 4                  0.0          0.0               0.0             0.0    0.0   
 
    ...  founded_quarter_2012-Q3  founded_qua

In [8]:

# splitting the data into features and target variable
X_nn = data_nn.drop("success", axis=1)
y_nn = data_nn["success"]

# splitting the data into training and testing sets (80% train, 20% test)
X_train_nn, X_test_nn, y_train_nn, y_test_nn = train_test_split(X_nn, y_nn, test_size=0.2, random_state=42)

# scale the data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_nn)
X_test_scaled = scaler.transform(X_test_nn)

X_train_scaled.shape, X_test_scaled.shape


((17873, 210), (4469, 210))

This is sklearn MLPClassifer

In [9]:
# identify columns with NaN values in the training set
#nan_columns = X_train_nn.columns[X_train_nn.isnull().any()].tolist()

# identify columns with infinite values in the training set
#infinite_columns = X_train_nn.columns[np.isinf(X_train_nn).any()].tolist()

#nan_columns, infinite_columns
# drop rows with missing values from the training and testing sets
X_train_dropped = X_train_nn.dropna()
y_train_dropped = y_train_nn[X_train_dropped.index]

X_test_dropped = X_test_nn.dropna()
y_test_dropped = y_test_nn[X_test_dropped.index]

# scale the data after dropping rows with missing values
X_train_scaled_dropped = scaler.fit_transform(X_train_dropped)
X_test_scaled_dropped = scaler.transform(X_test_dropped)

# define and train the neural network using MLPClassifier
mlp_dropped = MLPClassifier(hidden_layer_sizes=(25, 50), max_iter=200, activation='relu', solver='adam', random_state=42)
mlp_dropped.fit(X_train_scaled_dropped, y_train_dropped)

# evaluate the training accuracy after dropping rows with missing values
train_accuracy_dropped = mlp_dropped.score(X_train_scaled_dropped, y_train_dropped)

train_accuracy_dropped



/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


0.9767271662763466